Finding matching park with taxi zone

In [1]:
import geopandas as gpd
import pandas as pd
import folium 
import matplotlib.pyplot as plt
import mapclassify

manhattan_parks_gdf = gpd.read_file("manhattan_parks_geocoded.geojson")
#manhattan_taxi_zones_gdf = gpd.read_file("manhattan_taxi_zones.geojson")


In [2]:
manhattan_parks_gdf.columns

Index(['id', 'name', 'website', 'area', 'geometry'], dtype='object')

In [3]:
manhattan_parks_gdf=manhattan_parks_gdf.drop('area',axis=1)

In [4]:
busyness_taxizones= gpd.read_file("C:/Users/Justh/OneDrive/Documents/MScComputerScience/S3 Research Practicum/DataAnalysis/preprocessing/yellow_taxi/passengers_by_zone_datetime.geojson")

In [5]:
busyness_taxizones_sample = busyness_taxizones.sample(n=10000, random_state=42)

In [6]:
manhattan_parks_gdf= manhattan_parks_gdf.to_crs(epsg=32618)
busyness_taxizones_sample=busyness_taxizones_sample.to_crs(epsg=32618)

In [7]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString, MultiLineString

#defining a function to assign an area to Point geometries
def point_to_area(point, radius=10):
    # convert radius to the unit of the CRS, assuming radius is in meters for projected CRS
    return point.buffer(radius).area

# calculate the area for each geometry
def calculate_area(geometry):
    if isinstance(geometry, (Polygon, MultiPolygon)):
        return geometry.area
    elif isinstance(geometry, Point):
        return point_to_area(geometry)
    elif isinstance(geometry, (LineString, MultiLineString)):
        return 0  # LineString and MultiLineString do not have an area
    else:
        return 0

manhattan_parks_gdf['park_area'] = manhattan_parks_gdf['geometry'].apply(calculate_area)

print(manhattan_parks_gdf[['geometry', 'park_area']])


                                              geometry      park_area
0    POLYGON ((588697.294 4513735.907, 588660.844 4...    9506.859431
1    POLYGON ((583189.679 4505151.211, 583176.341 4...  696227.552870
2    MULTIPOLYGON (((590104.734 4522324.808, 590103...  521310.282769
3    MULTIPOLYGON (((584986.193 4507940.755, 584945...   36496.652767
4    MULTIPOLYGON (((590333.858 4524352.525, 590327...  281836.601376
..                                                 ...            ...
467                     POINT (585777.327 4513877.184)     313.654849
468                     POINT (585781.018 4513876.206)     313.654849
469                     POINT (585773.618 4513871.635)     313.654849
470                     POINT (585779.325 4513869.314)     313.654849
471                     POINT (585783.313 4513866.818)     313.654849

[472 rows x 2 columns]


In [8]:
# Calculate area taxi zones
busyness_taxizones_sample['taxi_zone_area'] = busyness_taxizones_sample.geometry.area

In [9]:
busyness_taxizones_sample.head(5)

,location_id,year,month,day,hour,passenger_count_number,zone,geometry,taxi_zone_area
91570,41,2022,12,4,14,17,Central Harlem,"MULTIPOLYGON (((588751.602 4518153.835, 588979...",1.340082e+06
7770,114,2022,8,10,8,1,Greenwich Village South,"MULTIPOLYGON (((585078.418 4508629.562, 585076...",4.408437e+05
55143,229,2022,4,4,20,38,Sutton Place/Turtle Bay North,"MULTIPOLYGON (((587651.653 4512095.775, 587604...",5.958605e+05
7033,114,2022,3,18,21,49,Greenwich Village South,"MULTIPOLYGON (((585078.418 4508629.562, 585076...",4.408437e+05
51520,211,2022,4,2,12,18,SoHo,"MULTIPOLYGON (((584328.619 4508134.642, 584298...",3.725053e+05


In [10]:
manhattan_parks_gdf.head(5)

,id,name,website,geometry,park_area
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431
1,relation/2389631,Governors Island,https://www.nps.gov/gois/index.htm,"POLYGON ((583189.679 4505151.211, 583176.341 4...",696227.552870
2,relation/3690816,Highbridge Park,https://www.nycgovparks.org/parks/highbridge-p...,"MULTIPOLYGON (((590104.734 4522324.808, 590103...",521310.282769
3,relation/4099553,Sara D. Roosevelt Park,https://www.nycgovparks.org/parks/sara-d-roose...,"MULTIPOLYGON (((584986.193 4507940.755, 584945...",36496.652767
4,relation/5625630,Fort Tryon Park,https://www.nycgovparks.org/parks/fort-tryon-park,"MULTIPOLYGON (((590333.858 4524352.525, 590327...",281836.601376


In [11]:
manhattan_parks_gdf.crs

<Projected CRS: EPSG:32618>
Name: WGS 84 / UTM zone 18N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 78°W and 72°W, northern hemisphere between equator and 84°N, onshore and offshore. Bahamas. Canada - Nunavut; Ontario; Quebec. Colombia. Cuba. Ecuador. Greenland. Haiti. Jamaica. Panama. Turks and Caicos Islands. United States (USA). Venezuela.
- bounds: (-78.0, 0.0, -72.0, 84.0)
Coordinate Operation:
- name: UTM zone 18N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
busyness_taxizones_sample.columns

Index(['location_id', 'year', 'month', 'day', 'hour', 'passenger_count_number',
       'zone', 'geometry', 'taxi_zone_area'],
      dtype='object')

In [13]:
busyness_taxizones_sample=busyness_taxizones_sample.rename(columns={'location_id':'taxi_zone_id','passenger_count_number':'taxi_zone_busyness'})

In [14]:
busyness_taxizones_sample=busyness_taxizones_sample.rename(columns={'zone':'taxi_zone'})

In [15]:
manhattan_parks_gdf.columns

Index(['id', 'name', 'website', 'geometry', 'park_area'], dtype='object')

In [16]:
manhattan_parks_gdf=manhattan_parks_gdf.rename(columns={'id':'park_id','name':'park_name'})

#### Spatial join the parks with each taxi zone

In [17]:
# Perform a spatial join to find parks within each taxi zone
parks_with_zones = gpd.sjoin(manhattan_parks_gdf, busyness_taxizones_sample, how='inner', predicate='intersects')

print(parks_with_zones[['park_name', 'taxi_zone_id']])


                                  park_name taxi_zone_id
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
..                                      ...          ...
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142

[76724 rows x 2 columns]


In [18]:
parks_with_zones.head()

,park_id,park_name,website,geometry,park_area,index_right,taxi_zone_id,year,month,day,hour,taxi_zone_busyness,taxi_zone,taxi_zone_area
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,16393,140,2022,1,4,12,32,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,17895,140,2022,7,14,16,36,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18347,140,2022,9,8,0,16,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18247,140,2022,9,2,18,1,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18159,140,2022,8,5,10,29,Lenox Hill East,692556.6829


Calculating remaining taxi zone area

In [19]:
# Calculate the remaining area of the taxi zone
parks_with_zones['remaining_taxi_zone_area'] = parks_with_zones['taxi_zone_area'] - parks_with_zones['park_area']

# Calculate the proportion of the taxi zone area that is covered by the park
parks_with_zones['park_proportion'] = parks_with_zones['park_area'] / parks_with_zones['taxi_zone_area']

# Calculate the busyness score for the park
parks_with_zones['park_busyness'] = parks_with_zones['park_proportion'] * parks_with_zones['taxi_zone_busyness']

# Display the resulting DataFrame
print(parks_with_zones[['park_name', 'park_id','park_area','taxi_zone_area','park_proportion', 'taxi_zone_busyness','park_busyness']])


                                  park_name           park_id    park_area  \
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
..                                      ...               ...          ...   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   

     taxi_zone_area  park_proportion  taxi_zone_busyness  park_

In [1]:
parks_with_zones.head(20)

NameError: name 'parks_with_zones' is not defined

In [21]:
parks_with_zones.to_csv('park_testing.csv',index=False)

In [22]:
parks_with_zones

,park_id,park_name,website,geometry,park_area,index_right,taxi_zone_id,year,month,day,hour,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,16393,140,2022,1,4,12,32,Lenox Hill East,692556.68290,683049.823468,0.013727,0.439270
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,17895,140,2022,7,14,16,36,Lenox Hill East,692556.68290,683049.823468,0.013727,0.494179
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18347,140,2022,9,8,0,16,Lenox Hill East,692556.68290,683049.823468,0.013727,0.219635
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18247,140,2022,9,2,18,1,Lenox Hill East,692556.68290,683049.823468,0.013727,0.013727
0,relation/1751075,John Jay Park,N/A,"POLYGON ((588697.294 4513735.907, 588660.844 4...",9506.859431,18159,140,2022,8,5,10,29,Lenox Hill East,692556.68290,683049.823468,0.013727,0.398089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,POINT (585783.313 4513866.818),313.654849,20573,142,2022,1,19,19,11,Lincoln Square East,708820.86973,708507.214881,0.000443,0.004868
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,POINT (585783.313 4513866.818),313.654849,21761,142,2022,4,29,14,43,Lincoln Square East,708820.86973,708507.214881,0.000443,0.019028
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,POINT (585783.313 4513866.818),313.654849,21782,142,2022,4,4,19,36,Lincoln Square East,708820.86973,708507.214881,0.000443,0.015930
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,POINT (585783.313 4513866.818),313.654849,21254,142,2022,12,7,17,14,Lincoln Square East,708820.86973,708507.214881,0.000443,0.006195


In [23]:
parks_with_zones.shape

(76724, 17)

In [24]:
park_busyness_details= parks_with_zones.groupby(['park_id','park_name','taxi_zone_id','geometry'])['park_busyness'].sum().reset_index()


In [25]:
type(park_busyness_details)

pandas.core.frame.DataFrame

In [26]:
# Set the geometry column explicitly
park_busyness_details= gpd.GeoDataFrame(park_busyness_details, geometry='geometry')

In [27]:
park_busyness_details

,park_id,park_name,taxi_zone_id,geometry,park_busyness
0,node/11457630561,Kowsky Plaza,13,POINT (583037.173 4507235.738),0.656036
1,node/1203709404,Sherman Creek,127,POINT (591160.550 4523886.465),0.022707
2,node/3369582883,"103 W 62nd St, New York, NY 10023, USA",142,POINT (585777.522 4513869.759),1.665136
3,node/3369582884,"625 Jaffe Dr, New York, NY 10023, USA",142,POINT (585780.485 4513881.407),1.665136
4,node/3369582885,"103 W 62nd St, New York, NY 10023, USA",142,POINT (585782.078 4513872.865),1.665136
...,...,...,...,...,...
495,way/973521995,Ahearn Park,232,"POLYGON ((585841.666 4507605.118, 585842.707 4...",0.181663
496,way/976724943,"1120 Cooper Sq, New York, NY 10003, USA",113,"POLYGON ((585192.359 4509213.937, 585188.264 4...",0.651830
497,way/985281628,Seward Park High School - Community Playground,232,"POLYGON ((585341.645 4507674.799, 585395.681 4...",2.046101
498,way/990936691,Pier 42 Upland Park,232,"POLYGON ((586058.724 4507144.473, 586025.750 4...",8.608015


In [28]:
park_busyness_details.geom_type.value_counts()

Polygon         420
Point            40
MultiPolygon     33
LineString        7
Name: count, dtype: int64

In [29]:
park_busyness_details.explore("park_busyness", legend=True)

In [30]:
central_park_details= parks_with_zones[parks_with_zones['park_name']=='Central Park']
central_park_details

,park_id,park_name,website,geometry,park_area,index_right,taxi_zone_id,year,month,day,hour,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,95820,43,2022,5,26,16,5,Central Park,3.556719e+06,142980.04328,0.9598,4.7990
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,96416,43,2022,9,2,17,7,Central Park,3.556719e+06,142980.04328,0.9598,6.7186
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,95485,43,2022,3,25,21,6,Central Park,3.556719e+06,142980.04328,0.9598,5.7588
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,94969,43,2022,11,24,22,2,Central Park,3.556719e+06,142980.04328,0.9598,1.9196
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,95908,43,2022,6,1,22,6,Central Park,3.556719e+06,142980.04328,0.9598,5.7588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,95545,43,2022,4,1,17,5,Central Park,3.556719e+06,142980.04328,0.9598,4.7990
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,96430,43,2022,9,21,14,14,Central Park,3.556719e+06,142980.04328,0.9598,13.4372
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,96140,43,2022,7,23,22,5,Central Park,3.556719e+06,142980.04328,0.9598,4.7990
279,way/427818536,Central Park,https://www.centralparknyc.org,"POLYGON ((587923.358 4517112.634, 587921.532 4...",3.413739e+06,95826,43,2022,5,27,13,5,Central Park,3.556719e+06,142980.04328,0.9598,4.7990
